In [1]:
import numpy as np
import simpy
import random

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [29]:
SEED = 930
UNIFORM_INTERVAL = [4.17, 5.37]
NUM_SERVER = 2
SERVER_UTILIZATIONS_PHASE1 = [0.6, 0.7, 0.8, 0.9]
SERVER_SERVICE_MEAN_PHASE1 = [5.725,6.679,7.634,8.588]
SERVER_UTILIATION_PHASE2 = 0.8
NUMBER_OF_CUSTOMERS = 100


In [30]:
phase_1_lib = {"num_server":NUM_SERVER,"arr_interval":UNIFORM_INTERVAL,"util":SERVER_UTILIZATIONS_PHASE1,"serv_mean":SERVER_SERVICE_MEAN_PHASE1}
phase_2_lib = {"num_server":NUM_SERVER,"arr_interval":UNIFORM_INTERVAL,"util":SERVER_UTILIATION_PHASE2,"serv_mean":7.634}

In [31]:
class Jobs(object):
    def __init__(self, name, env, operator , run_):
        self.env = env
        self.name = name
        self.operator = operator
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        self.run = run_
    
    
    def call(self):
        print('%s initiated at %g' % (self.name, self.env.now))
 
        with self.operator.request() as req:
            yield req
            print('%s is assigned to an operator at %g' % (self.name, self.env.now))
            self.run.queue_w_times.append(self.env.now - self.arrival_t)
            yield self.env.process(self.progress_job())
            print('%s is done at %g' % (self.name, self.env.now))
            
    def progress_job(self):
        duration = random.expovariate(self.run.serv_rate)
        yield self.env.timeout(duration)
        self.run.service_times.append(duration)
        
        

In [32]:
class Run():
    def __init__(self,arr_rate,serv_rate):
        self.seed = random.random()*425
        self.run = False
        self.arr_rate = arr_rate
        self.serv_rate = 1/serv_rate
        self.jobs=[]
        self.service_times=[]
        self.queue_w_times=[]
        self.env = simpy.Environment()
        self.operator = simpy.Resource(self.env, capacity = 1)
    
    def run_sim(self):
        random.seed(self.seed)
        self.env.process(self.jobs_generator())
        self.env.run()
        self.run = True
        
    def show_run(self):
        if self.run!=True:
            self.run_sim()
        
        


    def jobs_generator(self):
        for i in range(NUMBER_OF_CUSTOMERS):
            yield self.env.timeout(random.expovariate(self.arr_rate))
            job = Jobs('Job %s' %(i+1), self.env, self.operator,self)
            self.jobs.append(job)  

In [33]:
class Phase():
    def __init__(self,lib,num_runs):
        self.num_server = lib["num_server"]
        self.arr_int = 1/((lib["arr_interval"][0]+lib["arr_interval"][1])/2)
        self.util = lib["util"]
        self.serv_mean = lib["serv_mean"]
        self.init_runs(num_runs)


    def init_runs(self,num_runs):
        self.runs = []
        for util_no in range(len(self.util)):
            for i in range(num_runs):
                obj = Run(self.arr_int,self.serv_mean[util_no])
                self.runs.append(obj)

    def run_phase(self):
        for run in self.runs:
            run.run_sim()
        print("total run :",len(self.runs)," ")


In [34]:
phase_1 = Phase(phase_1_lib,10)

In [35]:
phase_1.run_phase()

Cust 1 initiated a call at 3.01079
Cust 1 is assigned to an operator at 3.01079
Cust 1 is done at 3.05785
Cust 2 initiated a call at 4.76007
Cust 2 is assigned to an operator at 4.76007
Cust 2 is done at 5.45473
Cust 3 initiated a call at 16.0101
Cust 3 is assigned to an operator at 16.0101
Cust 3 is done at 17.067
Cust 4 initiated a call at 22.9138
Cust 4 is assigned to an operator at 22.9138
Cust 5 initiated a call at 31.0808
Cust 6 initiated a call at 31.1201
Cust 4 is done at 36.3138
Cust 5 is assigned to an operator at 36.3138
Cust 5 is done at 41.2168
Cust 6 is assigned to an operator at 41.2168
Cust 7 initiated a call at 41.6549
Cust 6 is done at 41.9692
Cust 7 is assigned to an operator at 41.9692
Cust 7 is done at 44.0189
Cust 8 initiated a call at 67.7827
Cust 8 is assigned to an operator at 67.7827
Cust 8 is done at 68.5954
Cust 9 initiated a call at 71.7824
Cust 9 is assigned to an operator at 71.7824
Cust 10 initiated a call at 73.9259
Cust 9 is done at 77.7731
Cust 10 is 